In [1]:
%install '.package(path: "$cwd/YOLOv4")' YOLOv4
import TensorFlow
import YOLOv4

Installing packages:
	.package(path: "/notebooks/YOLOv4")
		YOLOv4
With SwiftPM flags: []
Working in: /tmp/tmpkpxzqswe/swift-install
[1/4] Compiling YOLOv4 YOLOv4.swift
/notebooks/YOLOv4/Sources/YOLOv4.swift:10:27: warning: redundant conformance constraint 'Features': 'Differentiable'
public func |> <Features: Differentiable, Func: Layer>(left: Func.Input, right:Func) -> Func.Output where Features == Func.Input {
                          ^
/notebooks/YOLOv4/Sources/YOLOv4.swift:10:49: note: conformance constraint 'Features': 'Differentiable' implied here
public func |> <Features: Differentiable, Func: Layer>(left: Func.Input, right:Func) -> Func.Output where Features == Func.Input {
                                                ^
[2/5] Merging module YOLOv4
[3/6] Wrapping AST for YOLOv4 for debugging
[4/6] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[5/7] Merging module jupyterInstalledPackages
[6/7] Wrapping AST for jupyterInstalledPackages for debugging
[7/7]

In [2]:
var x = Tensor<Float>(randomNormal: [1,416,416,3])
var yolo = YOLOv4<Float>(cls:80)
print((x |> yolo))

[[[  3.3264469e-06,  -2.1009526e-05,  -2.0867712e-05, ...,  -1.4366923e-07,
     -3.077563e-08,  -1.8900792e-08],
  [  -3.226304e-05,  -1.0938505e-05,  -2.9647788e-05, ...,  -9.1466696e-08,
     -6.797588e-08,    7.038947e-08],
  [ -1.1371455e-05,   1.0602875e-05,   2.3008903e-05, ...,  -2.0322915e-07,
    -1.9045471e-07,  -1.5011473e-07],
  ...,
  [   8.421959e-06,   1.8719751e-05,   5.4644624e-07, ...,   1.1846028e-07,
     1.9455237e-07,   1.2247384e-07],
  [  -6.540129e-06,   2.1672126e-06,   -2.033667e-05, ...,  -1.2976294e-07,
      -7.89294e-08,  -5.6469522e-09],
  [  1.2360367e-05,   -3.488159e-06,  -1.3749217e-05, ...,    1.352265e-07,
     1.8675182e-08,    8.508319e-08]]]


In [3]:
import Dispatch
func time(_ function: () -> ()) {
    let start = DispatchTime.now()
    function()
    let end = DispatchTime.now()
    let nanoseconds = Double(end.uptimeNanoseconds - start.uptimeNanoseconds)
    let milliseconds = nanoseconds / 1e6
    // Could do fancier pretty-printing logic based on the order of magnitude.
    print("\(milliseconds) ms")
}

func time(re: Int, _ function: () -> ()) {
    time{
        for i in 1...re {
            function()
                        }
        }
  // Run it a few times and calculate mean and stdev.
}

In [9]:
time(re:10){
    var x = Tensor<Float>(randomNormal: [1,416,416,3])
    x |> yolo
           }

7780.0279 ms


In [28]:
import Foundation

func readFileToInt32 (path: URL?) -> [Int32] {
    var numbers: [Int32] = []
    if let data: Data = NSData(contentsOf: path!) as Data?  {
        var iter = data.makeIterator()
        while true {
            guard
                let b1 = iter.next(),
                let b2 = iter.next(),
            let b3 = iter.next(),
            let b4 = iter.next()
                else {
                    break
                }
            let num = Int32(b4) << 24 | Int32(b3) << 16 | Int32(b2) << 8 | Int32(b1)
            numbers.append(num)
        }
    }else{
       print ("in readFileToInt16...read file failed")
    }
    return numbers
}

var x = readFileToInt32(path: URL(string:"$cwd/yolov4.weights"))

Fatal error: You must link or load module FoundationNetworking to load non-file: URL content using String(contentsOf:…), Data(contentsOf:…), etc.: file /swift-base/swift-corelibs-foundation/Sources/Foundation/NSSwiftRuntime.swift, line 401
Current stack trace:
0    libswiftCore.so                    0x00007f3c7c57c160 swift_reportError + 50
1    libswiftCore.so                    0x00007f3c7c5eec20 _swift_stdlib_reportFatalErrorInFile + 115
2    libswiftCore.so                    0x00007f3c7c293cb5 <unavailable> + 1506485
3    libswiftCore.so                    0x00007f3c7c2938f7 <unavailable> + 1505527
4    libswiftCore.so                    0x00007f3c7c293e92 <unavailable> + 1506962
5    libswiftCore.so                    0x00007f3c7c2922e0 _assertionFailure(_:_:file:line:flags:) + 517
6    libFoundation.so                   0x00007f3c0d399304 <unavailable> + 6914820
7    libFoundation.so                   0x00007f3c0d222507 <unavailable> + 5379335
8    libFoundation.so              

: 

In [18]:
import Foundation

func floatValue(data: Data) -> Float {
    return Float(bitPattern: UInt32(bigEndian: data.withUnsafeBytes { $0.load(as: UInt32.self) }))
}

if let stream:InputStream = InputStream(fileAtPath: "yolov3.weights") {
            var buf:[UInt8] = [UInt8](repeating: 0, count: 4)
            stream.open()
            for i in 1...100 {
                let len = stream.read(&buf, maxLength: buf.count) 
                //print("len \(len)")
                buf.reverse()
                print(floatValue(data:Data(bytes: buf, count: 4)))
                
                if len < buf.count {
                    break
                }
            }
            stream.close()
        }

0.0
3e-45
0.0
8.540127e-38
0.0
-4.316885
-0.7578076
-2.1098018
1.7402638
1.4071269
-3.0952053
-0.38860837
0.75603795
1.982805
1.2893223
0.652888
2.6263633
2.3013082
-2.04827
-3.7340226
-2.0467598
3.845535
-1.0419698
-0.30135924
-0.35420752
-3.5354283
-2.628548
0.74821305
0.3917957
2.3627155
-1.7907225
2.5973146
-0.34963462
-2.6972923
-2.688753
0.99702346
-0.20098142
2.6223972
1.3536469
1.6286678
0.99905145
-1.0922571
2.6365428
0.9870672
1.6019626
0.8613433
-1.3214242
1.1380545
1.0997647
1.7826979
1.5409452
2.5057783
1.8394158
0.9597077
1.506216
1.1141889
1.1447448
1.7145146
-2.1283774
-1.0026759
-1.4917758
0.65230376
1.4994049
1.6686112
1.1225682
2.2276077
2.1827738
1.1739247
1.1685042
-0.5719402
-0.7738392
-0.28336832
0.39322987
0.007807612
-0.6109912
-0.16133149
-0.22186707
-0.29902607
0.117041886
-0.0055340216
0.017555995
-0.02504491
0.0042533753
0.42962
-0.44072402
-0.2520025
-0.21482663
-0.0024812848
-0.2174843
-0.78819096
0.4381482
-0.06644736
0.111937724
0.47471324
-0.006802341
